## Colombia Public Company Financial Information Crawler
##### Last Update: 2022-08-02
#### Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Python Version: Python 3.10.4 (ipykernel)
#### Chrome Version: Chrome 103.0.5060.134 (64-bit)
#### Chrome Driver Version: ChromeDriver 103.0.5060.134

#### Crawled Website
Investing.com MAIN: https://www.investing.com /
Investing.com Colombia: https://www.investing.com/equities/colombia

Description: This is public company financial statement web crawling code for investing.com, especially for companies in Colombia. investing.com is an unofficial stock information website for countries around the world. Even though it is an unofficial website, comparably, it has decent amount of information including key financial values. However, the server itself is very unstable that loading error occurs a lot, and data table format/language format is not standardized that a lot of exceptions are made(goes to failList). So one should be aware of that, and it is better to find alternative source for a long term development. Using excel format instead of csv could be a solution, which I have not tried so far. This code can be reused for other countries by editing initial website address and several page setting button XPATH locations - Colombia, Indonesia, Italy, Sinbgapore, and Spain is already written and uploaded. For crawling, as long as the website has not been modified, the code below must be run properly if and only if it is run in an order. Moreover, environment setting must be fixed based on user's computer/server setting and location.

### 1. Importing useful open source librabries - utilized BS4 and Selenium Web Driver for crawling

In [1]:
import bs4
import time
import csv 
import pandas as pd 
from platform import python_version
import requests
import lxml 
import xlrd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

### 2. Environment
#### - Chromdriver must be located in local/cloud PC folder.
#### - One can download proper version at: chromedriver.chromium.org/downloads
#### - Path should be modified based on local environment

In [3]:
cover = open('Colombia_2022_investing_2022-07-05_Chris.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

### 3. Main Crawler
#### - Uncollectable or unavailable data was collected as "" for convenience.
#### - Data Cleansing might be required after crawling sometiems, but it will not be tough as one knows how to use excel.
#### - Some directions and explanations are written as comment below.
#### - The order of append must be depend on the order of column name assigned above.
#### - Lots of try-except function is used to avoid error caused by non-existing pages or information.
#### - One also need to avoid ad and accept cookie banner that randomly appears.
#### - Run the crawler many times to improve the accuracy - Detail is written below main crawler.

In [ ]:
# Basic Setting
failList = []
info = []

# Get EURONEXT Brussels Stock Website
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/colombia')
# Avoid Ad
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select').click()
time.sleep(2)
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select/option[1]').click()
time.sleep(4)

# Get each company's link using href in a tag
text = driver.page_source
time.sleep(2)
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='cross_rate_markets_stocks_1')
time.sleep(2)
all_atag_maintable = maintable.find_all('a')
print(maintable)
company_links = []

# We have to process the collected data again because of their data structure
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)

# Main Data Collection code
for sub in company_links:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(base_url + sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가코드
        info.append("COL") 
        info.append("Colombia") 
        info.append("Colombia") 
        info.append("UTC-05:00") 
        info.append("남아메리카")
        info.append("351281000000 USD") 
        info.append("50372424") 
        info.append("라틴아메리카")

        #기업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("COLHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Colombian Securities Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Colombian Securities Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Colombian Securities Exchange. They are providing products/services in Colombia, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Colombia"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Colombia, South America"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Colombia"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Colombia, South America"
                post = "N/A"
            except:
                address1 = "Colombia"
                address2 = "Colombia, South America"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. COP")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. COP")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. COP")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. COP")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. COP")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. COP")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. COP")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. COP")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. COP")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. COP")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. COP")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. COP")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. COP")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. COP")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. COP")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. COP")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. COP")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. COP")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. COP")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. COP")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. COP")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("ESP")
        info.append("Spanish")
        info.append("Espanol")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("BVC")
        info.append("Colombian Securities Exchange")
        info.append("Bolsa de Valores de Colombia")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "COP")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Colombia")
        info.append("Colombia, South America")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("COP")
        info.append("Colombian Peso")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-05")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList.append(base_url + sub)
            
print(failList)

### 4. Dealing with FailList
#### - Because Investing.com causes loading time error due to unstable server status, it is better to run the code, at least with companies in failList, for several times or run with two or more computers. That will increase the sucess rate much higher than running the code once. When running a new code, make sure to change the name of saving file so that you don't replace the original file.
#### - There are many ways to deal with elements failList: retry(if caused by server loading status), ignore(if caused by actually non-existing data), manually add(if only few exist), find why error has occurred(if massive).